# Regression Final Project

### Importing Libs

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from csv import writer
import re
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',100)

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor()

from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

**Defination for Scrabing**

In [2]:
def house_scraping(url,csv):
    #url = "https://www.hepsiemlak.com/antalya-kiralik"
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content,'html.parser')
    lists = soup.find_all('li',class_="listing-item")
    
    with open(csv,'a',encoding='utf8',newline='') as f:
        csv_writer = writer(f)
        header = ['Description','Weekend','Area','Age','Floor','Price']
        csv_writer.writerow(header)
        for list in lists:
            description = list.find('header',class_ = 'list-view-header').text.replace('\n','')
            weekend = list.find('span',class_ = 'celly houseRoomCount').text.replace('\n','')
            area = list.find('span',class_ = 'celly squareMeter list-view-size').text.replace('\n','')
            age = list.find('span',class_ = 'celly buildingAge').text.replace('\n','')
            floor = list.find('span',class_ = 'celly floortype').text.replace('\n','')
            price = list.find('span',class_ = 'list-view-price').text.replace('\n','')
            info = [description,weekend,area,age,floor,price]
            csv_writer.writerow(info)

**Definiton for writing data to csv file**

In [3]:
def county(name,page):

    for i in range(page):
        if i == 0:
            house_scraping(url=f"https://www.hepsiemlak.com/{name}-kiralik",csv=f"hepsiemlak_{name}.csv")
        else:
            house_scraping(url=f"https://www.hepsiemlak.com/{name}-kiralik?page={i+1}",csv=f"hepsiemlak_{name}.csv")


# Lets Scraping !

In [4]:
#county("istanbul",10)#Write the city name and the number of advertisements on the site
df = pd.read_csv("hepsiemlak_istanbul.csv")
df

,Description,Weekend,Area,Age,Floor,Price
0,NaN,5 + 1,330 m2,45 Yaşında,Bahçe Katı,155.000 TL
1,NaN,5 + 2,388 m2,10 Yaşında,NaN,77.500 TL
2,NaN,1 + 1,108 m2,5 Yaşında,Ara Kat,38.000 TL
3,NaN,4 + 2,520 m2,100 Yaşında,NaN,230.000 TL
4,NaN,5 + 2,385 m2,15 Yaşında,NaN,67.500 TL
...,...,...,...,...,...,...
1595,nüans'tan ortaköy'de i...,1 + 1,60 m2,40 Yaşında,Bahçe Katı,9.500 TL
1596,sisli near osmanbey me...,2 + 1,90 m2,15 Yaşında,3. Kat,25.000 TL
1597,imajplus'dan doğa şehi...,1 + 1,130 m2,Sıfır Bina,7. Kat,35.000 TL
1598,prizma'dan bebek'te mu...,3 + 1,200 m2,50 Yaşında,2. Kat,90.000 TL


# EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  1540 non-null   object
 1   Weekend      1600 non-null   object
 2   Area         1600 non-null   object
 3   Age          1600 non-null   object
 4   Floor        1480 non-null   object
 5   Price        1600 non-null   object
dtypes: object(6)
memory usage: 75.1+ KB


In [6]:
df.head()

,Description,Weekend,Area,Age,Floor,Price
0,NaN,5 + 1,330 m2,45 Yaşında,Bahçe Katı,155.000 TL
1,NaN,5 + 2,388 m2,10 Yaşında,NaN,77.500 TL
2,NaN,1 + 1,108 m2,5 Yaşında,Ara Kat,38.000 TL
3,NaN,4 + 2,520 m2,100 Yaşında,NaN,230.000 TL
4,NaN,5 + 2,385 m2,15 Yaşında,NaN,67.500 TL


In [8]:
df.shape

(1600, 6)

In [9]:
df.isnull().sum()

Description     60
Weekend          0
Area             0
Age              0
Floor          120
Price            0
dtype: int64

# Feature Engineering

In [10]:
df=df.drop_duplicates()

In [11]:
df['Area'] = df['Area'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
#Removing everything just the digits are revealing on data

In [12]:
df.head()
#Ohh we forgot the 2 from m^2 

,Description,Weekend,Area,Age,Floor,Price
0,NaN,5 + 1,3302.0,45 Yaşında,Bahçe Katı,155.000 TL
1,NaN,5 + 2,3882.0,10 Yaşında,NaN,77.500 TL
2,NaN,1 + 1,1082.0,5 Yaşında,Ara Kat,38.000 TL
3,NaN,4 + 2,5202.0,100 Yaşında,NaN,230.000 TL
4,NaN,5 + 2,3852.0,15 Yaşında,NaN,67.500 TL


In [13]:
miss=df["Area"].isnull()
miss[miss==True],

In [ ]:
df.drop(24,axis=0,inplace=True)

In [14]:
df["Area"]=df.Area/10
df["Area"]=df["Area"].astype(int)

In [15]:
df.head()

,Description,Weekend,Area,Age,Floor,Price
0,NaN,5 + 1,330,45 Yaşında,Bahçe Katı,155.000 TL
1,NaN,5 + 2,388,10 Yaşında,NaN,77.500 TL
2,NaN,1 + 1,108,5 Yaşında,Ara Kat,38.000 TL
3,NaN,4 + 2,520,100 Yaşında,NaN,230.000 TL
4,NaN,5 + 2,385,15 Yaşında,NaN,67.500 TL


In [16]:
df.isnull().sum()

Description    10
Weekend         0
Area            0
Age             0
Floor          32
Price           0
dtype: int64

In [17]:
df['Floor'].value_counts()

                      2. Kat             55
                      3. Kat             50
                      1. Kat             48
                      4. Kat             46
                      Ara Kat            28
                      Bahçe Katı         25
                      5. Kat             23
                      Yüksek Giriş       19
                      Giriş Katı         18
                      En Üst Kat         13
                      6. Kat             12
                      9. Kat             11
                      8. Kat             10
                      11. Kat             9
                      7. Kat              9
                      21 ve üzeri         7
                      10. Kat             6
                      12. Kat             4
                      Zemin               3
                      Villa Katı          3
                      16. Kat             3
                      14. Kat             3
                      Çatı Katı 

In [18]:
df['Floor'].replace({"                      Bodrum ve Zemin":0,"                      Zemin":0,"                      Yarı Bodrum":0,"                      Ara Kat":2,"                      Bahçe Katı":0,"                      Yüksek Giriş":1,"                      Giriş Katı":0,"                      En Üst Kat":3,"                      Villa Katı":3,"                      Çatı Katı":3},inplace=True)

In [19]:
df['Floor'] = df['Floor'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [20]:
df['Floor'].fillna((df['Floor'].mean()), inplace=True)

In [21]:
df['Floor']=df['Floor'].astype(int)

In [22]:
df['Floor'].value_counts()

3     102
2      84
1      70
0      50
4      46
5      23
6      12
9      11
8      10
11      9
7       9
21      7
10      6
12      4
14      3
16      3
18      2
13      2
19      1
15      1
Name: Floor, dtype: int64

In [23]:
df['Age'].value_counts()

Sıfır Bina     42
30 Yaşında     37
25 Yaşında     32
35 Yaşında     27
10 Yaşında     24
40 Yaşında     23
15 Yaşında     22
20 Yaşında     20
5 Yaşında      18
4 Yaşında      17
1 Yaşında      14
3 Yaşında      12
2 Yaşında      10
38 Yaşında     10
6 Yaşında      10
22 Yaşında     10
11 Yaşında      9
8 Yaşında       9
16 Yaşında      9
26 Yaşında      8
50 Yaşında      8
27 Yaşında      7
33 Yaşında      7
45 Yaşında      7
32 Yaşında      6
12 Yaşında      6
7 Yaşında       6
21 Yaşında      5
9 Yaşında       5
18 Yaşında      5
55 Yaşında      4
14 Yaşında      3
24 Yaşında      3
28 Yaşında      3
23 Yaşında      3
31 Yaşında      2
17 Yaşında      2
13 Yaşında      2
19 Yaşında      2
100 Yaşında     1
42 Yaşında      1
39 Yaşında      1
29 Yaşında      1
36 Yaşında      1
47 Yaşında      1
Name: Age, dtype: int64

In [24]:
df['Age'].replace("Sıfır Bina",0,inplace=True)

In [25]:
df['Age'] = df['Age'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [26]:
df['Age']=df['Age'].astype(int)

In [27]:
df['Age'].value_counts()

0      42
30     37
25     32
35     27
10     24
40     23
15     22
20     20
5      18
4      17
1      14
3      12
2      10
38     10
6      10
22     10
11      9
8       9
16      9
26      8
50      8
27      7
33      7
45      7
32      6
12      6
7       6
21      5
9       5
18      5
55      4
14      3
24      3
28      3
23      3
31      2
17      2
13      2
19      2
100     1
42      1
39      1
29      1
36      1
47      1
Name: Age, dtype: int64

In [28]:
df['Price'].value_counts()

              10.000              TL    24
              15.000              TL    22
              8.000              TL     18
              25.000              TL    17
              12.000              TL    17
                                        ..
              7.200              TL      1
              34.490              TL     1
              21.000              TL     1
              28.000              TL     1
              82.000              TL     1
Name: Price, Length: 123, dtype: int64

In [29]:
df['Price'] = df['Price'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
df['Price']=df['Price'].astype(int)

In [30]:
df['Price'].value_counts().keys().sort_values(ascending=True)

Int64Index([  2750,   3000,   3100,   3500,   4000,   4500,   4850,   5000,
              5500,   5900,
            ...
            100000, 110000, 120000, 135000, 150000, 155000, 200000, 210000,
            230000, 255000],
           dtype='int64', length=123)

In [31]:
df.head()

,Description,Weekend,Area,Age,Floor,Price
0,NaN,5 + 1,330,45,0,155000
1,NaN,5 + 2,388,10,3,77500
2,NaN,1 + 1,108,5,2,38000
3,NaN,4 + 2,520,100,3,230000
4,NaN,5 + 2,385,15,3,67500


In [32]:
df.isnull().sum()

Description    10
Weekend         0
Area            0
Age             0
Floor           0
Price           0
dtype: int64

In [33]:
df["Weekend"].value_counts()

2 + 1     163
3 + 1     154
1 + 1      56
4 + 1      24
5 + 1      12
4 + 2      12
Stüdyo      8
5 + 2       6
3 + 2       6
6 + 2       4
2 + 2       3
7 + 1       2
6 + 1       2
3 + 3       1
14 + 3      1
6 + 3       1
Name: Weekend, dtype: int64

In [34]:
df['Weekend'] = df['Weekend'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
df["Weekend"].fillna(000.0,inplace=True)

In [35]:
df["Weekend"]=df["Weekend"].astype(int)


In [36]:
df.head()

,Description,Weekend,Area,Age,Floor,Price
0,NaN,51,330,45,0,155000
1,NaN,52,388,10,3,77500
2,NaN,11,108,5,2,38000
3,NaN,42,520,100,3,230000
4,NaN,52,385,15,3,67500


In [37]:
df.drop("Description",axis=1,inplace=True)

In [38]:
df["Weekend"]=df["Weekend"]*2
df["Area"]=df["Area"]*2

In [39]:
df=pd.get_dummies(df)

# Lets Predict!!!

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [41]:
lr = LinearRegression()
R=Ridge()
L=Lasso()
E=ElasticNet(l1_ratio=0.8)
ETR=ExtraTreeRegressor()
GBR=GradientBoostingRegressor()
KNR=KNeighborsRegressor(n_neighbors=10,weights="uniform")

In [42]:
def algo_test(x,y):
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree','XGBRegressor']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [43]:
x=df.drop("Price",axis=1)
y=df.Price

In [44]:
algo_test(x,y)

,R_Squared,RMSE,MAE
KNeighborsRegressor,0.561963,25213.072164,13306.263736
Linear,0.533253,26026.208858,13538.231023
Lasso,0.533253,26026.218142,13538.223022
Ridge,0.533253,26026.228590,13538.206133
ElasticNet,0.533119,26029.960206,13533.882499
Extra Tree,0.439997,28507.907333,12983.021978
XGBRegressor,0.372328,30181.212902,13256.055036
Gradient Boosting,0.362506,30416.430852,13961.598068
Decision Tree,0.093682,36266.923814,16187.857143
